In [7]:
import re 
p = re.compile("'([0-9]+)'")

In [8]:
import re 
import requests 
from bs4 import BeautifulSoup
def get_ticker(company):
    text_url = "https://dart.fss.or.kr/dsae001/search.ax"
    payload = {
    "startDate": "",     "endDate": "",
    "currentPage": 1,     "maxResults": 45,
    "maxLinks": 10,     "sort": "",
    "series": "",     "selectKey": "",
    "searchIndex": "",     "textCrpCik": "",
    "autoSearch": True,     "businessCode": "all",
    "bsnRgsNo": "",     "bsnRgsNo_1": "",
    "bsnRgsNo_2": "",     "bsnRgsNo_3": "",
    "crpRgsNo": "",     "textCrpNm": company,
    "corporationType": "all"
    }
    r = requests.post(text_url, data=payload)
    bs = BeautifulSoup(r.text)
    result = {x.text.strip():p.findall(x['href'])[0] for x in bs.find_all('a')[:-1]}
    return result[company]

In [9]:
get_ticker("삼성전자")

'00126380'

In [10]:
def get_statement(company, company_code, start_date, end_date):
    data = {
    "currentPage": 1,     "maxResults": 15,
    "maxLinks": 10,     "sort": "date",
    "series": "desc",     "textCrpCik": company_code,
    "lateKeyword": "",    "keyword": "",     "reportNamePopYn": "",
    "textkeyword": "",     "businessCode": "all",     "autoSearch": "N",
    "option": "corp",
    "textCrpNm": company,
    "reportName": "",    "tocSrch": "",
    "textPresenterNm": "",
    "startDate": start_date,
    "endDate":  end_date,
    "decadeType": "",    "finalReport": "recent",    "businessNm": "전체",
    "corporationType": "all",    "closingAccountsMonth": "all",
    "reportName2": "",    "tocSrch2": "",
    "publicType": [
        "A001", "A002", "A003", "A005", "A004",
        "B001", "B002", "B003",
        "C001", "C002", "C003", "C004", "C005", "C006", "C007", "C008", "C009", "C010", "C011",
        "D001", "D004", "D003", "D002", "D005",
        "E001", "E002", "E003", "E004", "E005", "E006", "E007", "E008", "E009",
        "F001", "F002", "F003", "F004", "F005",
        "G001", "G002", "G003",
        "H001", "H002", "H003", "H004", "H005", "H006"
        ]
    }
    url = "https://dart.fss.or.kr/dsab007/detailSearch.ax"
    return requests.post(url, data=data).text


In [11]:
rt = get_statement("삼성전자", "00126380", "20240101" , '20250731')

In [12]:
print(rt.strip())

<div class="tbListInner" >
<table class="tbList" summary="공시서류검색에 대한 번호, 공시대상회사, 보고서명, 제출인, 접수일자, 비고 등을 알리는 표입니다.">
	<caption>공시서류검색 목록</caption>
	<colgroup id="colgropup">
		<col style="width:6%">
		<col style="width:20%">
		<col style="width:auto">
		
		<col style="width:13%">
		<col style="width:11%">
		<col style="width:8%">
		
	</colgroup>
	<thead >
		<tr id="tr">
			<th scope="row"><label for="inpSample00">번호</label></th>
			<th scope="row"><label for="inpSample00">공시대상회사</label></th>
			<th scope="row"><label for="inpSample00">보고서명</label></th>
			
			<th scope="row"><label for="inpSample00">제출인</label></th>
			<th scope="row"><label for="inpSample00">접수일자</label></th>
			<th scope="row"><label for="inpSample00">비고</label></th>
			
		</tr>
	</thead>
	<tbody id="tbody">
	
		
			
				
				
				
				
				
				
				<tr>
					<td >
						1
					</td>
					<td class="tL">
					    <span class="innerWrap">
					    	<span class="tagCom_kospi" title="유가증권시장" style="cursor:default">유</sp

In [13]:
from langchain.agents import AgentType, initialize_agent, load_tools, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool

from pydantic import BaseModel, Field
class GetStatementInput(BaseModel):
    company: str = Field(..., description='회사 이름')
    company_code: str = Field(..., description='회사 코드')
    start_date: str = Field(..., description="YYYYMMDD 형식의 시작 날짜")
    end_date: str = Field(..., description="YYYYMMDD 형식의 종료 날짜")

In [14]:
from typing import Optional, Type
class GetStatementTool(BaseTool):
    name: str = 'GetStatementTool'             
    description: str = '기업의 공시 자료를 가져옵니다.' 
    
    args_schema: Type[BaseModel] = GetStatementInput

    def _run(self, company, company_code, start_date, end_date):
        return get_statement(company=company, company_code=company_code, start_date=start_date, end_date=end_date)

In [15]:
class GetTickerTool(BaseTool):
    name: str = 'GetTickerTool'                 
    description: str = 'Dart에서 사용하는 기업의 code 리턴 '

    def _run(self, company):
        return get_ticker(company=company)

In [16]:
tools = [GetTickerTool(), GetStatementTool()]
chat = ChatOpenAI(model = 'gpt-4o-2024-08-06')
agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

/tmp/ipykernel_717426/731457596.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model = 'gpt-4o-2024-08-06')
/tmp/ipykernel_717426/731457596.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ a

In [19]:
result = agent.run("당신은 지금부터 애널리스트입니다. 삼성전자의 20240101부터 20241231까지 공시 정보를 활용해서 분석 리포트를 자세히 작성할 것. 그리고 투자 의견에 대한 내용도 결론에 넣을 것")




> Entering new AgentExecutor chain...

Invoking: `GetTickerTool` with `{'company': '삼성전자'}`


00126380
Invoking: `GetStatementTool` with `{'company': '삼성전자', 'company_code': '00126380', 'start_date': '20240101', 'end_date': '20241231'}`













<div class="tbListInner" >
<table class="tbList" summary="공시서류검색에 대한 번호, 공시대상회사, 보고서명, 제출인, 접수일자, 비고 등을 알리는 표입니다.">
	<caption>공시서류검색 목록</caption>
	<colgroup id="colgropup">
		<col style="width:6%">
		<col style="width:20%">
		<col style="width:auto">
		
		<col style="width:13%">
		<col style="width:11%">
		<col style="width:8%">
		
	</colgroup>
	<thead >
		<tr id="tr">
			<th scope="row"><label for="inpSample00">번호</label></th>
			<th scope="row"><label for="inpSample00">공시대상회사</label></th>
			<th scope="row"><label for="inpSample00">보고서명</label></th>
			
			<th scope="row"><label for="inpSample00">제출인</label></th>
			<th scope="row"><label for="inpSample00">접수일자</label></th>
			<th scope="row"><label for="inpSample00">비고</label></th>
			

In [20]:
print(result)

# 삼성전자 2024년 공시 정보 분석 리포트

## 1. 개요

본 리포트는 삼성전자의 2024년 공시 자료를 기반으로 기업의 주요 활동과 재무 상태를 분석하고, 이에 따른 투자 의견을 제시합니다. 삼성전자는 반도체, 스마트폰, 디스플레이 등 다양한 사업 분야에서 글로벌 시장을 선도하는 기업입니다.

## 2. 공시 자료 요약

2024년 삼성전자는 총 182건의 공시를 등록하였으며, 그 중 일부는 다음과 같습니다:

- **임원 및 주요 주주 특정 증권 등 소유 상황 보고서**: 다수의 임원 및 주요 주주와 관련된 소유 상황 보고서가 2024년 12월에 집중적으로 발표되었습니다. 이는 기업 지배구조와 관련된 주요 변화를 시사할 수 있습니다.

- **주식 등의 대량 보유 상황 보고서**: 삼성물산으로부터 2024년 12월에 제출된 보고서는 삼성전자 주식의 대량 보유 상태를 명확히 하고 있습니다. 이는 기업 간 관계와 주식시장 내에서의 전략적 움직임을 나타냅니다.

## 3. 주요 분석

### 3.1. 지배구조 변화

2024년 하반기에 집중적으로 보고된 임원 및 주요 주주의 소유 상황 보고서는 삼성전자의 지배구조에 변화가 있을 가능성을 제시합니다. 이는 미래의 전략적 방향 및 경영진의 의사 결정에 영향을 미칠 수 있습니다.

### 3.2. 주식 보유 및 시장 반응

삼성물산의 대량 주식 보유 보고서는 삼성전자와 삼성물산 간의 긴밀한 관계를 보여줍니다. 이는 두 기업 간의 전략적 제휴 및 협력 가능성을 시사하며, 시장에서는 이를 긍정적으로 평가할 가능성이 높습니다.

### 3.3. 재무 상태 및 사업 전망

공시 자료를 통해 삼성전자의 재무 상태는 안정적이며, 주요 사업 분야에서의 지속적인 혁신과 성장이 예상됩니다. 반도체 및 스마트폰 시장에서의 기술적 우위는 삼성전자의 경쟁력을 더욱 강화할 것입니다.

## 4. 투자 의견

### 결론

삼성전자는 글로벌 시장에서의 위치와 혁신 역량을 고려할 때, 장기적인 투자 가치가 높은 기업으로 평가됩니다. 임원